# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [ ]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [15]:
from prettytable import PrettyTable
import pandas as pd 
table = PrettyTable(['id', 'minutes'])
recipes = pd.read_csv('recipes_sample.csv')
random_recipes = recipes.sample(n=5)[['id', 'minutes']]
for i, row in random_recipes.iterrows():    
  table.add_row([row['id'], row['minutes']])
print(table)

+--------+---------+
|   id   | minutes |
+--------+---------+
| 213876 |    45   |
| 73851  |    40   |
| 105722 |    15   |
| 398156 |   105   |
| 136011 |    45   |
+--------+---------+


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

In [36]:
import pandas as pd
import xml.etree.ElementTree as ET

def show_info(id):
    # Read recipes 
    recipes = pd.read_csv('recipes_sample.csv')
    recipe = recipes.loc[recipes['id'] == id].iloc[0]

    name = recipe['name']
    author_id = recipe['contributor_id']
    minutes = recipe['minutes']

    # Read steps
    steps_tree = ET.parse('steps_sample.xml')
    steps_root = steps_tree.getroot()
    steps = []
    for recipe_elem in steps_root.findall('recipe'):
        if recipe_elem.findtext('id') == str(id):
            for step_elem in recipe_elem.findall('steps/step'):
                steps.append(step_elem.text.strip())

    output = f'"{name}"\n\n'
    for i, step in enumerate(steps, start=1):
        output += f'{i}. {step}\n'
    output += '----------\n'
    output += f'Автор: {author_id}\n'
    output += f'Среднее время приготовления: {minutes} минут\n'
    return output

print(show_info(170895))

"leeks and parsnips  sauteed or creamed"

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [44]:
import re 
import xml.etree.ElementTree as ET

def get_step_id(recipe_id):
  lst=[]
  text=".//recipe/[id ='"+ str(recipe_id)+"']"
  a=steps_root.findall(text)
  for steps in a[0].iter('step'):
    lst.append(steps.text)
  return lst

steps_tree = ET.parse('steps_sample.xml')
steps_root = steps_tree.getroot()
steps= get_step_id(25082)

for i in steps:
  match= re.search(r'\d+\s(hour|hours|minute|minutes)',i )
  if match: 
    print(match[0])

20 minute
10 minute
2 hour
10 minute
20 minute


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [48]:
from numpy import rec
recipes = pd.read_csv('recipes_sample.csv')
patt='this [\w\s]*, (?:but|but)'
r= recipes.loc[recipes.description.str.match(patt)==True]
print(f"количество рецептов : {r.shape[0]}\n")
for i in range(1,4):
  print(f"рецепт {i}: {r.description.values[i]}") 

количество рецептов : 131

рецепт 1: this is kind of similar to some of the other versions out there, but it is the best and easiest i have found
рецепт 2: this is a moist, buttery apple cake.  the dough is best made the night before, put in the fridge and then filled and baked the next morning.
рецепт 3: this pie does not have very many ingredients in it, but it is ohhh so good. i don't know why it's called tears on your pillow pie. funny name, but great pie.


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [63]:
import re 
import xml.etree.ElementTree as ET

steps_tree = ET.parse('steps_sample.xml')
steps_root = steps_tree.getroot()

for recipe in steps_root.findall('recipe'):
    if recipe.find('id').text == '72367':
        steps = [step.text for step in recipe.findall("step")]
        for i in steps:
            if bool(re.findall(r'\d+ /\ d+ ', i )):
                i = re.sub(r' / ', '/', i)
            print(i)

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [69]:

import nltk
import xml.etree.ElementTree as ET
nltk.download('punkt')
steps_tree = ET.parse('steps_sample.xml')
steps_root = steps_tree.getroot()

unik=set()
words=0

for item in steps_root.iter('step'):
  str=item.text
  lw=nltk.word_tokenize(str)
  for item1 in lw:
    if item1.isalpha()==True:
      unik.add(item1.lower())
      words+=1
print("всего слов:", words, "уникальный слов:", len(unik))






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


всего слов: 2734377 уникальный слов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [83]:
import pandas as pd
import nltk
nltk.download('punkt')

recipes = pd.read_csv('recipes_sample.csv')
recipes = recipes.dropna(subset=['description'])
def split_sentences(text):
    return nltk.sent_tokenize(text)

recipes['sentences'] = recipes['description'].apply(split_sentences)

print(recipes.head(5))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                       name     id  minutes  contributor_id  \
0     george s at the cove  black bean soup  44123       90           35193   
1        healthy for them  yogurt popsicles  67664       10           91970   
2              i can t believe it s spinach  38798       30            1533   
3                      italian  gut busters  35173       45           22724   
4  love is in the air  beef fondue   sauces  84797       25            4470   

    submitted  n_steps                                        description  \
0  2002-10-25      NaN  an original recipe created by chef scott meska...   
1  2003-07-26      NaN  my children and their friends ask for my homem...   
2  2002-08-29      NaN            these were so go, it surprised even me.   
3  2002-07-27      NaN  my sister-in-law made these for us at a family...   
4  2004-02-23      4.0  i think a fondue is a very romantic casual din...   

   n_ingredients                                          sent

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [86]:
import pandas as pd
import nltk

nltk.download('averaged_perceptron_tagger')
recipes = pd.read_csv('recipes_sample.csv')

for i in recipes.index:
  if recipes.id[i]==241106:
    ind=i
a=nltk.pos_tag([i for i in nltk.tokenize.word_tokenize(recipes.name[ind])])
s1,s2='',''
for i,j in dict(a).items():
  s1+=f'{j:^{len(i)+1}}'
  s2+=i+' '
print(s1)
print(s2)






[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


   JJ      NNS   IN     NNS     VBP   JJ    CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
